In [ ]:
import numpy as np
import tensorflow as tf
import keras

from importlib import reload
import src.text_generation as text_gen


In [ ]:

path_to_file = keras.utils.get_file(
        'shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
        )
print(path_to_file)

with open(path_to_file) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:148])

shakespeare_tensor = tf.constant([shakespeare_text])
vocab = list(set(shakespeare_text.lower().strip()))
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(shakespeare_text.lower())
n_tokens = len(tokenizer.word_index)

In [10]:
from keras.models import load_model

# Load the model from the saved path
model = load_model("models/shakespeare_gru_1.keras")
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, None, 128)      │        64,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 39)       │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 209,783 (819.47 KB)

 Trainable params: 69,927 (273.15 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 139,856 (546.32 KB)

In [11]:
text = "to be "
sequence = np.array(tokenizer.texts_to_sequences([text])[0])
print(sequence)
print(tokenizer.sequences_to_texts([sequence]))
sequence = sequence - 1
print(sequence)

[ 3  4  1 22  2  1]
['t o   b e  ']
[ 2  3  0 21  1  0]


In [12]:
inp = tf.one_hot(sequence, depth=n_tokens)
print(inp.shape)
inp_tf = tf.reshape(inp, (1, 6, n_tokens))
print(inp_tf.shape)


(6, 39)
(1, 6, 39)


In [13]:
y = model.predict(inp_tf)
print(y.shape)
prediction = y[0, -1, :]
print(prediction.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
(1, 6, 39)
(39,)


2025-04-25 09:05:21.968485: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [14]:
char_id = np.argmax(prediction) + 1
print(char_id)

sequence = np.append(sequence + 1, char_id)
print(sequence)
tokenizer.sequences_to_texts([sequence])

5
[ 3  4  1 22  2  1  5]


['t o   b e   a']

In [21]:
reload(text_gen)
one_hot_fn = lambda x: tf.one_hot(x, depth=n_tokens)
sequence = np.array(tokenizer.texts_to_sequences([text])[0])
sequence = sequence - 1
text_gen.next_token(sequence, model, transform=one_hot_fn, argmax=False, temperature=10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
prediction: (39,)


np.int64(14)

In [ ]:
from src.text_generation import next_token

next_token(sequence, model, transform=one_hot_fn, argmax=False, temperature=10)

In [23]:
reload(text_gen)

text_gen.generate("to be ", model, tokenizer, n_chars=50, temperature=0.3, argmax=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: (39,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
p

't o   b e   a n d   t h e   w o r l d   i n   t h e   s e r v i c e \n a n d   s e e m   t h e   s e r v i c e  '